### Black Scholes Model

They showed that combining risky assets can elimiante risk itself.

Modern Portfolio Theory is about including several stocks in a portfolio in order to reduce unsystematic risk, specific risk to individual stocks that can be diversified away.

Capm is about systematic risk or market that can not be diversified away. all about beta. 

Market neutral strategies and delta hedging and pairs trading can eliminate all risk. 

V(S,t,...) value of an option is a function of various parameters
V (s,t, o, u, E, T, r)
s,t - stock price and time are variable
o,u - volatility and mean are parameters associated with the stock price
E,T  - strike price and expiry are parameters associated with the given option
r - parameter associated with risk free rate

Call option will rise in value if the underlying asset rises and falls if the asset falls

Option and underlying stock as a positive correlation for call options

Option and underlying stock has a negative correlation for put options

we can exploit these correlations to end up with a rather special portfolio

TT = V(S,t) - dS
     long pos. short pos
    
The special TT portfolio contains a long position in the V(S,t) option and a short optionn in the S(t) underlying stock

what is d? we short just some quantity of the underlying

### Dynamic Delta Hedging

Assumption the underlying asset follows a log-normal random walk
https://en.wikipedia.org/wiki/Log-normal_distribution

dS = u S dt + o S dx
      deterministic + stochastic (winer)

what about the change in the portfolios value

The TT portfolio changes from time t to t+dt first of due to change in the V(S,t) option value + because of the change in the S(t) underlying asset

dTT = dV(S,t) - ddS

How to deal with these changes? With the help of Ito's lemma

<img src="images/black_scholes_1.png" />


any elimination in randomness is called hedging
delta hedging the perfect elimination of rosk because of exploiting the coorrelation between two assets 
it is an example of dynamic hedging

### No Arbitrage Principal

<img src="images/time_value_of_money.png" />

<img src="images/no_arbitrage_principal.png" />

Arbitrage when the invest can make riskless money

If we can get a greater return withthe delta hedged portfolio we could borrow money from the bank and pay interest at rate r invest that money into risk free option/stock portfolio and we could make a profit

usually we can assume there are no arbitrage opportunities or if there are they exist for just a short period of time

<img src="images/black_scholes_2.png" />

<img src="images/black_scholes_3.png" />

### The Greeks
Of Option Pricing

Delta d

d = dV / dS

velocity of underyling price

Delta - The delta of an option or portfolio is the sensitivity of the option or portfolio to the underlying

the delta of a portfolio is the sunm of deltas or all individual positions

it is important when dealing with delta hedging  we can eliminate risk if we make sure delta is equals to the derivative

Gamma r

r = d^2V / ds^2

acceleration of the underlying price

Gamma - The Gamma of an option or portfolio is the second derivative of the position with respect to the underlying. It is basically the sensitivity of the Delta. Gamme is a measure of how often a position must be rehedged in order to maintaini a delta-neutral position.

Delta-neutral it means the portfolio value remains unchanged when small changes occur in the value of the underlying. 

Theta o

o = dV / dt

Theta - the rate of change of the option price with time

Vega 

dV / do

Vega - some models rely heavily on the volatility so it may be important to monitor the change according to volatility

<img src="images/black_scholes_greeks.png" />



In [32]:
import numpy as np
from numpy.lib.scimath import log as nlog, sqrt as nsqrt
import pandas as pd
from scipy import log,exp, sqrt, stats

In [33]:
# underlying strike price
s0 = 100
# strike price
E=100
#expiry
T =1 # 1 year = 365 days
#Risk free rate of return
rf= 0.05
#Volatility of the underlying stock
sigma =0.2
# Number of iterations in MC simulation
iterations = 10000

<img src="images/black_scholes_3.png"/>

Pairs trading strategy - Developed at Morgan Stanley

Combining long and short positions in a pair of highly correlated financial instruments such as stocks
it is a form of statistical arbitrage developed in the 80s

if the correlation between the two stocks weakens investors should long the underperforming stock and short the outperforming stock -> because they are expected to converge in the future

In [34]:
# Calculate Call
d1 = (nlog(s0/E)+(rf+0.5*sigma*sigma)*T)/(sigma*nsqrt(T))
d2 = d1 - sigma*nsqrt(T)
d1,d2

(0.35000000000000003, 0.15000000000000002)

In [35]:
# Cumulative distribution function of the given RV.
call_price = s0*stats.norm.cdf(d1)-E*np.exp(-rf*T)*stats.norm.cdf(d2)
put_price = -s0*stats.norm.cdf(-d1)+E*np.exp(-rf*T)*stats.norm.cdf(-d2)
call_price, put_price

(10.450583572185565, 5.573526022256971)

### Monte Carlo Simulation
<img src="images/black_scholes_mc.png" />
<img src="images/black_scholes_mc_2.png" />

In [36]:
class OptionPricing:
    def __init__(self, s0,E,T,rf,sigma,iterations):
        self.s0 = s0
        self.E = E
        self.T = T
        self.rf = rf
        self.sigma = sigma
        self.iterations = iterations
    
    def call_option_simulation(self):
        # we have 2 columns first with 0s
        # the second column will store the payoff
        #we need the first column of 0s
        # the payoff function is max(0,S-E) for call options
        option_data = np.zeros([self.iterations,2])
        
        #dimensions: 1 dim array with as many items as iterations
        rand = np.random.normal(0,1,[1,self.iterations])
        
        # equation for the S(t) stock price
        stock_price = self.s0*np.exp(self.T*(self.rf-0.5*self.sigma**2)+ self.sigma*np.sqrt(self.T)*rand)
        
        # we need S-E because we have to calculate the max(S-E,0)
        option_data[:,1] = stock_price - self.E
        
        #average for the Monte-Carlo method
        # np.amax() returns the max(0,S-E) according to the forumla
        average = np.sum(np.amax(option_data,axis=1))/float(self.iterations)
        
        # have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average
    
    def put_option_simulation(self):
        # we have 2 columns first with 0s
        # the second column will store the payoff
        #we need the first column of 0s
        # the payoff function is max(E-S,0) for put options
        option_data = np.zeros([self.iterations,2])
        
        #dimensions: 1 dim array with as many items as iterations
        rand = np.random.normal(0,1,[1,self.iterations])
        
        # equationn for the S(t) stock price
        stock_price = self.s0*np.exp(self.T*(self.rf-0.5*self.sigma**2)+ self.sigma*np.sqrt(self.T)*rand)
        
        # we need S-E because we have to calculate the max(S-E,0)
        option_data[:,1] = self.E - stock_price
        
        #average for the Monte-Carlo method for put
        # np.amax() returns the max(E-S,0) according to the forumla
        average = np.sum(np.amax(option_data,axis=1))/float(self.iterations)
        
        # have to use the exp(-rT) discount factor
        return np.exp(-1.0*self.rf*self.T)*average
        
        

In [37]:

o = OptionPricing(s0,E,T,rf,sigma,iterations)
sim_call_price = o.call_option_simulation()
sim_call_price

10.460503699469522

In [38]:
sim_put_price = o.put_option_simulation()
sim_put_price

5.609667675849664